In [53]:
# import os
# import shutil
# import random

# # Linking the paths to the dataset directory
# training_data_directory = 'D:\AIT736\Project\Dataset\Training'
# validation_data_directory = 'D:\AIT736\Project\Dataset\Validation'

# # Creating a validation folder
# os.makedirs(validation_data_directory, exist_ok=True)
# for class_name in os.listdir(training_data_directory):
#     os.makedirs(os.path.join(validation_data_directory, class_name), exist_ok=True)

# # Assigning the percentage for validation
# validation_split = 0.2

# # Iterating over each folder representing a class in the training dataset
# for class_name in os.listdir(training_data_directory):
#     class_path = os.path.join(training_data_directory, class_name)
#     images = os.listdir(class_path)
    
#     random.shuffle(images)
#     validation_size = int(len(images) * validation_split)
#     validation_images = images[:validation_size]
    
#     # Moving the selected images to validation folder
#     for img in validation_images:
#         source = os.path.join(class_path, img)
#         destination = os.path.join(validation_data_directory, class_name, img)
#         shutil.move(source, destination)

# print("Validation set created successfully!")


Validation set created successfully!


In [ ]:
import os
import shutil
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.preprocessing import image

# Assigning the paths
training_data_directory = 'D:/AIT736/Project/Dataset/Training'
validation_data_directory = 'D:/AIT736/Project/Dataset/Validation'
others_data_directory = 'D:/AIT736/Project/Dataset/Others_AIT736'

# Creating Validation folder
os.makedirs(validation_data_directory, exist_ok=True)
for class_name in os.listdir(training_data_directory):
    os.makedirs(os.path.join(validation_data_directory, class_name), exist_ok=True)

# We now add the images in 'Others_AIT736' to train and test
os.makedirs(os.path.join(training_data_directory, 'Others_AIT736'), exist_ok=True)
os.makedirs(os.path.join(validation_data_directory, 'Others_AIT736'), exist_ok=True)

# Assigning the percentage for validation
validation_split = 0.2

# Move part of "Others_AIT736" data to validation
other_images = os.listdir(others_data_directory)
random.shuffle(other_images)
validation_size = int(len(other_images) * validation_split)
validation_images = other_images[:validation_size]
training_images = other_images[validation_size:]

# Move images to training and validation folders
for img in training_images:
    shutil.copy(os.path.join(others_data_directory, img), os.path.join(training_data_directory, 'Others_AIT736', img))
for img in validation_images:
    shutil.copy(os.path.join(others_data_directory, img), os.path.join(validation_data_directory, 'Others_AIT736', img))


In [55]:
# We assign Image Data Generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
train_generator = train_datagen.flow_from_directory(
    training_data_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    validation_data_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 3927 images belonging to 5 classes.


Found 1103 images belonging to 5 classes.


In [56]:
# Create a separate ImageDataGenerator for testing without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the test generator
test_generator = test_datagen.flow_from_directory(
    'D:\AIT736\Project\Dataset\Testing',  
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2353 images belonging to 5 classes.


In [57]:
# Model setup using VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

# Updating the final output layer with 5 neurons with "Others_AIT736" class
predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the VGG16 base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │     6,422,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,138,757 (80.64 MB)

 Trainable params: 6,424,069 (24.51 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [60]:
# Training the model
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 233s 2s/step - accuracy: 0.7180 - loss: 0.7342 - val_accuracy: 0.8833 - val_loss: 0.3430
Epoch 2/20
  1/122 ━━━━━━━━━━━━━━━━━━━━ 3:52 2s/step - accuracy: 0.7812 - loss: 0.4667

c:\Users\bhart\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


122/122 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.7812 - loss: 0.4667 - val_accuracy: 0.8667 - val_loss: 0.3840
Epoch 3/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 197s 2s/step - accuracy: 0.8096 - loss: 0.5326 - val_accuracy: 0.9035 - val_loss: 0.2911
Epoch 4/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8438 - loss: 0.4630 - val_accuracy: 0.8000 - val_loss: 0.2709
Epoch 5/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 171s 1s/step - accuracy: 0.8281 - loss: 0.4498 - val_accuracy: 0.9164 - val_loss: 0.2548
Epoch 6/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8750 - loss: 0.4488 - val_accuracy: 1.0000 - val_loss: 0.3001
Epoch 7/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 335s 3s/step - accuracy: 0.8313 - loss: 0.4633 - val_accuracy: 0.9108 - val_loss: 0.2516
Epoch 8/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9375 - loss: 0.2501 - val_accuracy: 0.9333 - val_loss: 0.2184
Epoch 9/20
122/122 ━━━━━━━━━━━━━━━━━━━━ 336s 3s/step - accuracy: 0.8418 - loss: 0.4187 - val_accuracy: 0.9145 - 

In [61]:
# Fine-tune the model by unfreezing specific layers in VGG16
for layer in base_model.layers:
    if layer.name.startswith('block5'):
        layer.trainable = True
    else:
        layer.trainable = False

In [62]:
# Recompile model for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [74]:
# Training the model after fine-tuning
epochs = 10
history_fine = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
)


Epoch 1/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 193s 2s/step - accuracy: 0.9366 - loss: 0.1898 - val_accuracy: 0.9320 - val_loss: 0.1530
Epoch 2/10
  1/122 ━━━━━━━━━━━━━━━━━━━━ 2:33 1s/step - accuracy: 0.9688 - loss: 0.1804

c:\Users\bhart\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9688 - loss: 0.1804 - val_accuracy: 0.9333 - val_loss: 0.1819
Epoch 3/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.9321 - loss: 0.2025 - val_accuracy: 0.9439 - val_loss: 0.1470
Epoch 4/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9375 - loss: 0.1417 - val_accuracy: 0.9333 - val_loss: 0.1667
Epoch 5/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 200s 2s/step - accuracy: 0.9359 - loss: 0.1815 - val_accuracy: 0.9412 - val_loss: 0.1503
Epoch 6/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9062 - loss: 0.1455 - val_accuracy: 0.8667 - val_loss: 0.1377
Epoch 7/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 198s 2s/step - accuracy: 0.9480 - loss: 0.1638 - val_accuracy: 0.9531 - val_loss: 0.1343
Epoch 8/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9375 - loss: 0.2347 - val_accuracy: 1.0000 - val_loss: 0.0121
Epoch 9/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 0.9502 - loss: 0.1478 - val_accuracy: 0.9430 - 

In [76]:
# Loading and preprocessing an image for testing
def predict_image_class(image_path, model, threshold=0.7):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Predicting the class probabilities
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions)
    confidence = np.max(predictions)

    # Checking if confidence meets threshold
    if confidence < threshold:
        return "This is not a brain MRI scan."
    else:
        class_labels = list(train_generator.class_indices.keys())
        return f"Predicted class: {class_labels[predicted_class]} with confidence {confidence}"

In [77]:
# Evaluating the model on test data
test_data_directory = 'D:/AIT736/Project/Dataset/Testing'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_directory,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy}")
print(f"Test Loss: {test_loss}")

Found 2353 images belonging to 5 classes.


c:\Users\bhart\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


74/74 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.9501 - loss: 0.1381
Test Accuracy: 0.9443264007568359
Test Loss: 0.15648360550403595


In [11]:
# Saving the model
# model.save('brain_tumor_detection_model_with_others1.keras')

In [59]:
# model.save('brain_tumor_detection_model_with_others1.h5')

In [30]:
# Example
example_image_path = 'D:/AIT736/Project/Dataset/Testing/Others_AIT736/6_5.png'
print(predict_image_class(example_image_path, model, threshold=0.7))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
Predicted class: Others_AIT736 with confidence 0.9999929666519165


In [67]:
class_indices = train_generator.class_indices
class_labels = {v: k for k, v in class_indices.items()}
print(class_labels)

{0: 'Others_AIT736', 1: 'glioma', 2: 'meningioma', 3: 'notumor', 4: 'pituitary'}
